In [1]:

import numpy as np 
import matplotlib.pyplot as plt

# librería con SVM
from sklearn import svm

# Carga el conjunto de datos y algunas funciones adicionales
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Librería cuántico (Qibo)
import qibo
from qibo import gates
from qibo.models import Circuit

#Uso de backend de numpy en Qibo
qibo.set_backend("numpy")


[Qibo 0.1.12.dev0|INFO|2024-05-22 12:50:05]: Using numpy backend on /CPU:0


In [2]:
X, y = load_iris(return_X_y=True)

# Se selecciona solo la dos primeras clases del conjunto 
# de datos debido a que el clasificador es binario
X = X[:100]
y = y[:100]

# Escalar los datos a distribución normal estándar debido a la 
# codificación datos es periódica
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Escalar las etiquetas a -1 y 1 debido que es importante para SVM
y_scaled = 2 * (y - 0.5)

# Se divide en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [3]:
# kernel cuántico
def distancia_circuit(x,y):
    
    # cuatro cúbits: cuatro características
    n_qubits = len(X_train[0]) 
    n_shots = 8000

    # Create an empty circuit
    circuit = Circuit(n_qubits)

    # Codificación cuántica en puertas RY 
    for pos in range(n_qubits):
        circuit.add(gates.RY(pos, y[pos]))
        circuit.add(gates.RY(pos, x[pos]).dagger())
        circuit.add(gates.M(pos))

    # ejecución del circuito n veces (n_shots)
    result = circuit.execute(nshots=n_shots)
    # Devuelve las probabilidades analíticas
    return result.probabilities()
       
# Se devuelve el probabilidad del base computacional "0000"
kernel = lambda x1, x2: distancia_circuit(x1, x2)[0]

In [4]:
x = np.array([np.pi/2,np.pi/2,np.pi/2,np.pi/2])
kernel(x,x)

1.0

In [5]:
def kernel_matrix(A, B):
    return np.array([[kernel(a, b) for b in B] for a in A])

In [6]:
# Inicialización
qsvm = svm.SVC(kernel=kernel_matrix)

print("Entrenando...")
qsvm = qsvm.fit(X_train, y_train)

print("Entrenado!")

Entrenando...
Entrenado!


In [7]:
#test
print("Precisión con test...")

predictions = qsvm.predict(X_test)
accuracy_score(predictions, y_test)

Precisión con test...


1.0

In [8]:
modelclasico = svm.SVC(kernel="linear")
modelclasico.fit(X_train, y_train)

print("Precisión con test...")

predictions = modelclasico.predict(X_test)
accuracy_score(predictions, y_test)


Precisión con test...


1.0